In [10]:
import logging
import pandas as pd
from sklearn.model_selection import train_test_split
import os.path
from os import path
import pandas as pd
import numpy as np
import base64
import re

In [11]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast

# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

In [12]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [13]:
file_name = "poems_with_new_tags.zip"
poem_df = pd.read_csv('poems_with_new_tags.zip', compression='zip', header=0, quotechar='"')


In [14]:
poems_info = poem_df.drop(['star','author_stars','tags','tags_list','new_tags','new_first_tag'], axis=1)

In [15]:
poems_info.head()

,content,dynasty,author,title,写物,劝勉,家庭,快乐,悲苦,政治,朋友,游玩
0,《吴都赋》云：“户藏烟浦，家具画船。”唯吴兴为然。春游之盛，西湖未能过也。己酉岁，予与萧时父...,宋代,姜夔,琵琶仙·《吴都赋》云：「户藏烟浦,0,0,0,0,0,1,0,1
1,《廿一史弹词》第三段说秦汉开场词滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。青山依旧在，几...,明代,杨慎,临江仙·滚滚长江东逝水,1,1,0,0,1,1,0,0
2,《水经》云：“彭蠡之口有石钟山焉。”郦元以为下临深潭，微风鼓浪，水石相搏，声如洪钟。是说也，...,宋代,苏轼,石钟山记,1,0,0,0,0,1,0,1
3,【序】辛亥之冬，予载雪诣石湖。止既月，授简索句，且征新声，作此两曲。石湖把玩不已，使工妓隶习...,宋代,姜夔,暗香疏影,0,1,0,0,0,0,0,0
4,〔一枝花〕 攀出墙朵朵花，折临路枝枝柳。花攀红蕊嫩，柳折翠条柔，浪子风流。凭着我折柳攀花手...,元代,关汉卿,【南吕】一枝花不伏老,1,0,0,1,1,0,0,0


In [16]:
OUTPUT_COLUMSN=['写物', '劝勉', '家庭', '快乐', '悲苦', '政治', '朋友', '游玩']

In [17]:
data, data_test = train_test_split(poems_info, test_size = 0.2)

In [18]:
# Name of the BERT model to use
model_name = 'bert-base-chinese'
# Max length of tokens
max_length = 100
# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [19]:
data.columns

Index(['content', 'dynasty', 'author', 'title', '写物', '劝勉', '家庭', '快乐', '悲苦',
       '政治', '朋友', '游玩'],
      dtype='object')

In [20]:
bert = transformer_model.layers[0]
# Build your model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}
# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)
# Then build your model output
#'写物', '劝勉', '家庭', '快乐', '悲苦', '政治', '朋友', '游玩'
object_ = Dense(units=len(data['写物'].value_counts()), 
                kernel_initializer=TruncatedNormal(stddev=config.initializer_range),
                name='object_')(pooled_output)
encourage_ = Dense(units=len(data['劝勉'].value_counts()), 
                   kernel_initializer=TruncatedNormal(stddev=config.initializer_range), 
                   name='encourage_')(pooled_output)
family_ = Dense(units=len(data['家庭'].value_counts()), 
                kernel_initializer=TruncatedNormal(stddev=config.initializer_range), 
                name='family_')(pooled_output)
happiness_ = Dense(units=len(data['快乐'].value_counts()), 
                   kernel_initializer=TruncatedNormal(stddev=config.initializer_range), 
                   name='happiness_')(pooled_output)
sadness_ = Dense(units=len(data['悲苦'].value_counts()), 
                 kernel_initializer=TruncatedNormal(stddev=config.initializer_range), 
                 name='sadness_')(pooled_output)
policy_ = Dense(units=len(data['政治'].value_counts()), 
                kernel_initializer=TruncatedNormal(stddev=config.initializer_range), 
                name='policy_')(pooled_output)
friend_ = Dense(units=len(data['朋友'].value_counts()), 
                kernel_initializer=TruncatedNormal(stddev=config.initializer_range), 
                name='friend_')(pooled_output)
travel_ = Dense(units=len(data['游玩'].value_counts()), 
                kernel_initializer=TruncatedNormal(stddev=config.initializer_range), 
                name='travel_')(pooled_output)

outputs = {'object_': object_, 'encourage_': encourage_, 'family_':family_, 'happiness_':happiness_,\
          'sadness_': sadness_, 'policy_': policy_, 'friend_': friend_, 'travel_': travel_}
# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')
# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 102267648   input_ids[0][0]                  
__________________________________________________________________________________________________
pooled_output (Dropout)         (None, 768)          0           bert[0][1]                       
__________________________________________________________________________________________________
encourage_ (Dense)              (None, 2)            1538        pooled_output[0][0]              
_________________________________________________________________________

In [21]:
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss = {'object_': CategoricalCrossentropy(from_logits = False), 
        'encourage_': CategoricalCrossentropy(from_logits = False), 
        'family_':CategoricalCrossentropy(from_logits = False), 
        'happiness_':CategoricalCrossentropy(from_logits = False),
        'sadness_': CategoricalCrossentropy(from_logits = False), 
        'policy_': CategoricalCrossentropy(from_logits = False), 
        'friend_': CategoricalCrossentropy(from_logits = False), 
        'travel_': CategoricalCrossentropy(from_logits = False)}
metric = {'object_': CategoricalAccuracy('accuracy'), 
        'encourage_': CategoricalAccuracy('accuracy'), 
        'family_':CategoricalAccuracy('accuracy'), 
        'happiness_':CategoricalAccuracy('accuracy'),
        'sadness_': CategoricalAccuracy('accuracy'), 
        'policy_': CategoricalAccuracy('accuracy'), 
        'friend_': CategoricalAccuracy('accuracy'), 
        'travel_': CategoricalAccuracy('accuracy')}
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)
# Ready output data for the model
##'写物', '劝勉', '家庭', '快乐', '悲苦', '政治', '朋友', '游玩'
object_y = to_categorical(data['写物'])
encourage_y = to_categorical(data['劝勉'])
family_y = to_categorical(data['家庭'])
happiness_y = to_categorical(data['快乐'])
sadness_y = to_categorical(data['悲苦'])
policy_y = to_categorical(data['政治'])
friend_y = to_categorical(data['朋友'])
travel_y =to_categorical(data['游玩'])
y_list = {'object_': object_y, 'encourage_': encourage_y, 'family_':family_y, 
          'happiness_':happiness_y,'sadness_': sadness_y, 'policy_': policy_y, 
          'friend_': friend_y, 'travel_': travel_y}
# Tokenize the input (takes some time)
x = tokenizer(
    text=data['content'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids']},
    y=y_list,
    validation_split=0.2,
    batch_size=64,
    epochs=1)

62/62 [==============================] - 1229s 20s/step - loss: 25.6963 - encourage__loss: 0.6507 - family__loss: 11.6110 - friend__loss: 1.2992 - happiness__loss: 8.0092 - object__loss: 1.1724 - policy__loss: 1.0803 - sadness__loss: 1.3046 - travel__loss: 0.5689 - encourage__accuracy: 0.7623 - family__accuracy: 0.0924 - friend__accuracy: 0.7449 - happiness__accuracy: 0.8373 - object__accuracy: 0.4429 - policy__accuracy: 0.4619 - sadness__accuracy: 0.5558 - travel__accuracy: 0.2198 - val_loss: 31.2061 - val_encourage__loss: 0.5262 - val_family__loss: 14.5178 - val_friend__loss: 0.4948 - val_happiness__loss: 13.1485 - val_object__loss: 0.6898 - val_policy__loss: 0.6781 - val_sadness__loss: 0.6688 - val_travel__loss: 0.4820 - val_encourage__accuracy: 0.7810 - val_family__accuracy: 0.0993 - val_friend__accuracy: 0.7953 - val_happiness__accuracy: 0.8628 - val_object__accuracy: 0.4217 - val_policy__accuracy: 0.4565 - val_sadness__accuracy: 0.5906 - val_travel__accuracy: 0.1894


In [23]:
# Ready test data
test_object_y = to_categorical(data_test['写物'])
test_encourage_y = to_categorical(data_test['劝勉'])
test_family_y = to_categorical(data_test['家庭'])
test_happiness_y = to_categorical(data_test['快乐'])
test_sadness_y = to_categorical(data_test['悲苦'])
test_policy_y = to_categorical(data_test['政治'])
test_friend_y = to_categorical(data_test['朋友'])
test_travel_y =to_categorical(data_test['游玩'])
test_x = tokenizer(
    text=data_test['content'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)



In [15]:
# Run evaluation
test_y_list = {'object_': test_object_y, 'encourage_': test_encourage_y, 'family_':test_family_y, 
          'happiness_':test_happiness_y,'sadness_': test_sadness_y, 'policy_': test_policy_y, 
          'friend_': test_friend_y, 'travel_': test_travel_y}
model_eval = model.evaluate(
    x={'input_ids': test_x['input_ids']},
    y=test_y_list
)

39/39 [==============================] - 102s 3s/step - loss: 4.6788 - encourage__loss: 0.6096 - family__loss: 0.3024 - friend__loss: 0.5025 - happiness__loss: 0.3848 - object__loss: 0.7259 - policy__loss: 0.8202 - sadness__loss: 0.7905 - travel__loss: 0.5428 - encourage__accuracy: 0.7488 - family__accuracy: 0.9059 - friend__accuracy: 0.8044 - happiness__accuracy: 0.8494 - object__accuracy: 0.6113 - policy__accuracy: 0.6195 - sadness__accuracy: 0.6080 - travel__accuracy: 0.7578


In [19]:
# model.save("multi-label-model")

AttributeError: 'Functional' object has no attribute 'save_model'

In [21]:
# loaded_model = load_model('multi-label-model')

In [24]:
prediction=model.predict(test_x['input_ids'])

In [23]:
type(prediction)

dict

In [28]:
prediction['encourage_'][9]

array([ 1.106571 , -1.1924883], dtype=float32)

In [9]:
data_test.head()

NameError: name 'data_test' is not defined